# 第7章: データベース

artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

### 60. KVSの構築

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

artists = []
with gzip.open('./data/chapter07/artist.json.gz', 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        artists.append(json.loads(line))
        
artists[:10]

[{'ended': True,
  'gid': '8972b1c1-6482-4750-b51f-596d2edea8b1',
  'id': 805192,
  'name': 'WIK▲N',
  'sort_name': 'WIK▲N',
  'tags': [{'count': 1, 'value': 'sillyname'}]},
 {'ended': True,
  'gid': 'b4f76788-7e6f-41b7-ac7b-dfb67f66282e',
  'id': 578352,
  'name': 'Gustav Ruppke',
  'sort_name': 'Gustav Ruppke',
  'type': 'Person'},
 {'ended': True,
  'gid': '49add228-eac5-4de8-836c-d75cde7369c3',
  'id': 371203,
  'name': 'Pete Moutso',
  'sort_name': 'Moutso, Pete',
  'type': 'Person'},
 {'ended': True,
  'gid': 'c112a400-af49-4665-8bba-741531d962a1',
  'id': 273232,
  'name': 'Zachary',
  'sort_name': 'Zachary'},
 {'ended': True,
  'gid': 'c42eed94-e233-44e2-82b8-3ed6dd9bf318',
  'id': 153193,
  'name': 'The High Level Ranters',
  'sort_name': 'High Level Ranters, The',
  'type': 'Group'},
 {'aliases': [{'name': 'Silhouettes', 'sort_name': 'Silhouettes'},
   {'name': 'The Sihouettes', 'sort_name': 'The Sihouettes'}],
  'area': 'United States',
  'begin': {'year': 1956},
  'end': {'

In [2]:
import redis

# redis サーバの設定
# ホスト名はDocke Composeで設定した名前を指定するとDocker側で名前解決してくれる
r = redis.StrictRedis(host='redis', port=6379, db=0)
pipe = r.pipeline()
pipe.flushall()
for i in artists:
    if 'area' in i.keys():
        # 名前の重複に対応するため名前とidをアンダーバーでつなげたものをキーとした
        pipe.set(i['name'] + "_" + str(i['id']), i['area'])

pipe.keys()
pipe.execute()[-1][:10]

[b'Arne Vinzon_741499',
 b'Richard Kirk_962529',
 b'No Way Sis_982638',
 b'heymoonshaker_897586',
 b'B.Lester_991148',
 b'Shannon Selberg_433232',
 b'Kubrak_758537',
 b'Power of Idea_1104176',
 b'Anna-Leena H\xc3\xa4rk\xc3\xb6nen_215590',
 b'\xe5\xb1\xb1\xe8\xb7\xaf\xe4\xb8\x80\xe5\xbf\x97_1073296']

### 61. KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [3]:
for i in r.keys(pattern='Oasis_*'):
    print(i.decode('utf8') + ": " + r.get(i).decode('utf8'))

Oasis_377879: United Kingdom
Oasis_20660: United Kingdom
Oasis_286198: United States


### 62. KVS内の反復処理

60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [4]:
len([i for i in r.keys() if r.get(i).decode('utf8') == "Japan"])

22821

### 63. オブジェクトを値に格納したKVS

KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [5]:
r_obj = redis.StrictRedis(host='redis', port=6379, db=1)
obj_pipe = r_obj.pipeline()
obj_pipe.flushall()
for i in artists:
    if 'tags' in i.keys():
        for j in i['tags']:
            obj_pipe.lpush(i['name'], j)

obj_pipe.keys()
obj_pipe.execute()[-1][:10]

[b'Lothar Koch',
 b'\xe6\xa1\x82\xe9\x9b\x80\xe3\x80\x85',
 b'\xe9\xbe\x8d\xe9\x96\x80\xe6\xb8\x95\xe9\x80\x8f\xe8\x8f\xaf',
 b'Skumju akme\xc5\x86i',
 b'Marc Fosset',
 b'Pepe Lienhard',
 b'The Low Countries',
 b'Jason Marbach',
 b'Looza',
 b'Totenkopf']

In [6]:
# タグ数と被タグ数
for i in r_obj.lrange('Totenkopf',0 ,-1):
    tag = json.loads(i.decode('utf8').replace("'", '"'))
    print("%s: %d" % (tag['value'], tag['count']))

rac: 1
twats: 1
nazi: 1
racist: 1


### 64. MongoDBの構築

アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [7]:
import pymongo

# mongodb へのアクセスを確立
mongo = pymongo.MongoClient('mongo', 27017)
# DBの作成
db = mongo.nlp100
# コレクションの作成
collection = db.artists

コレクションに `artists` のオブジェクトをinsert

In [8]:
# バルクインサートするぞい〜
collection.insert_many(artists)

### 65. MongoDBの検索

MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [9]:
# インタラクティブシェル使えって書いてあるけど、どうやったらnotebook内でインタラクティブシェル使えるのかわからないので省略
list(collection.find({"name": "Queen"}))

[{'_id': ObjectId('5a37be088d417a0015902a87'),
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}],
  'area': 'Japan',
  'ended': True,
  'gender': 'Female',
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'id': 701492,
  'name': 'Queen',
  'sort_name': 'Queen',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'type': 'Character'},
 {'_id': ObjectId('5a37be0a8d417a001590f133'),
  'aliases': [{'name': '女王', 'sort_name': '女王'}],
  'area': 'United Kingdom',
  'begin': {'date': 27, 'month': 6, 'year': 1970},
  'ended': True,
  'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3',
  'id': 192,
  'name': 'Queen',
  'rating': {'count': 24, 'value': 92},
  'sort_name': 'Queen',
  'tags': [{'count': 2, 'value': 'hard rock'},
   {'count': 1, 'value': '70s'},
   {'count': 1, 'value': 'queen family'},
   {'count': 1, 'value': '90s'},
   {'count': 1, 'value': '80s'},
   {'count': 1, 'value': 'glam rock'},
   {'count': 4, 'value': 'british'},
   {'count

### 66. 検索件数の取得

MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ

In [10]:
collection.find({"area": "Japan"}).count()

45642

### 67. 複数のドキュメントの取得

特定の（指定した）別名を持つアーティストを検索せよ．

In [11]:
# aliasesに別名が入ってる
list(collection.find({"aliases.name": "Silhouettes"}))

[{'_id': ObjectId('5a37bdfe8d417a001586f368'),
  'aliases': [{'name': 'Silhouettes', 'sort_name': 'Silhouettes'},
   {'name': 'The Sihouettes', 'sort_name': 'The Sihouettes'}],
  'area': 'United States',
  'begin': {'year': 1956},
  'end': {'year': 1993},
  'ended': True,
  'gid': 'ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b',
  'id': 101060,
  'name': 'The Silhouettes',
  'sort_name': 'Silhouettes, The',
  'type': 'Group'},
 {'_id': ObjectId('5a37d8d38d417a00a3e860e1'),
  'aliases': [{'name': 'Silhouettes', 'sort_name': 'Silhouettes'},
   {'name': 'The Sihouettes', 'sort_name': 'The Sihouettes'}],
  'area': 'United States',
  'begin': {'year': 1956},
  'end': {'year': 1993},
  'ended': True,
  'gid': 'ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b',
  'id': 101060,
  'name': 'The Silhouettes',
  'sort_name': 'Silhouettes, The',
  'type': 'Group'}]

### 68. ソート

"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [12]:
# tags.valueがdanceのもので絞り込み
dance_collection = collection.find({"tags.value": "dance"})

# rating.countを降順でソートし最初の10個を取ってくる
list(dance_collection.sort("rating.count", pymongo.DESCENDING))[:10]

[{'_id': ObjectId('5a37be0a8d417a0015920484'),
  'area': 'United States',
  'begin': {'date': 16, 'month': 8, 'year': 1958},
  'ended': True,
  'gender': 'Female',
  'gid': '79239441-bfd5-4981-a70c-55c3f15c1287',
  'id': 89,
  'name': 'Madonna',
  'rating': {'count': 26, 'value': 88},
  'sort_name': 'Madonna',
  'tags': [{'count': 1, 'value': 'dance-pop'},
   {'count': 1, 'value': 'electropop'},
   {'count': 1, 'value': 'tell me'},
   {'count': 1, 'value': 'pop and chart'},
   {'count': 1, 'value': 'multiple ipi'},
   {'count': 1, 'value': 'electronic'},
   {'count': 1, 'value': 'américain'},
   {'count': 1, 'value': 'usa'},
   {'count': 1, 'value': 'singer'},
   {'count': 1, 'value': 'chanteur'},
   {'count': 1, 'value': 'american'},
   {'count': 4, 'value': 'pop'},
   {'count': 1, 'value': 'greatest hits'},
   {'count': 1, 'value': 'dance'}],
  'type': 'Person'},
 {'_id': ObjectId('5a37d8df8d417a00a3f371fd'),
  'area': 'United States',
  'begin': {'date': 16, 'month': 8, 'year': 1958

### 69. Webアプリケーションの作成

ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．

_省略!!!!!!!!_